# Biomedical database tools from Biomni available via a enterprise gateway

In this notebook, you will use 30+ biomedical database tools from [Stanford Biomni](https://biomni.stanford.edu/about) made available via Bedrock AgentCore Gateway. The gateway has already been deployed for you in your AWS account. 
You will create a research agent using Strands that can search for relevant tools from the gateway and then query them to generate a response. 
## 1. Prerequisites

- Python 3.10 or later
- AWS account configured with appropriate permissions
- Access to the Anthropic Claude Sonnet 4 model on Amazon Bedrock
- Basic understanding of Python programming

In [1]:
%pip install -U boto3 strands-agents strands-agents-tools defusedxml httpx bedrock_agentcore_starter_toolkit

Looking in indexes: https://pypi.org/simple, https://plugin.us-east-1.prod.workshops.aws
  Using cached defusedxml-0.7.1-py2.py3-none-any.whl.metadata (32 kB)
Using cached defusedxml-0.7.1-py2.py3-none-any.whl (25 kB)
Note: you may need to restart the kernel to use updated packages.


## 2. Test the gateway to see which tools are available

To begin, we'll create a crendetials provider with AgentCore Identity to access the gateway. 

In [4]:
!python ../../../scripts/cognito_credentials_provider.py create --name researchapp-cp

🚀 Creating Cognito credential provider: researchapp-cp
📍 Region: us-west-2
📥 Fetching Cognito configuration from SSM...
✅ Retrieved client ID: 6u85e9r5ubsv96p9uph6km1p15
✅ Retrieved client secret: 1mhr***
✅ Issuer: https://cognito-idp.us-west-2.amazonaws.com/us-west-2_elMs8PQR5/.well-known/openid-configuration
✅ Authorization Endpoint: https://us-west-26e11baf0.auth.us-west-2.amazoncognito.com/oauth2/authorize
✅ Token Endpoint: https://us-west-26e11baf0.auth.us-west-2.amazoncognito.com/oauth2/token
⚙️  Creating OAuth2 credential provider...
✅ OAuth2 credential provider created successfully
   Provider ARN: arn:aws:bedrock-agentcore:us-west-2:942514891246:token-vault/default/oauth2credentialprovider/researchapp-cp
   Provider Name: researchapp-cp
🔐 Stored provider name in SSM: /app/researchapp/agentcore/cognito_provider
🎉 Cognito credential provider created successfully!
   Provider ARN: arn:aws:bedrock-agentcore:us-west-2:942514891246:token-vault/default/oauth2credentialprovider/resear

The gateway exposes Biomni's database tools through the Model Context Protocol (MCP). Note the database tools generate and execute dynamic REST API queries with independent Bedrock LLM invocations. You can see the code in `prerequisite/lambda-database`. 

In [1]:
from database_tools import get_gateway_access_token, get_all_mcp_tools_from_mcp_client, tool_search, tools_to_strands_mcp_tools
from utils import get_ssm_parameter
from mcp.client.streamable_http import streamablehttp_client
from strands import Agent
from strands_tools import current_time
from strands.models import BedrockModel
from strands.tools.mcp import MCPClient
import time
# Get gateway access token
jwt_token = get_gateway_access_token()
if not jwt_token:
    print("❌ Failed to get gateway access token")
    
# Get gateway endpoint
gateway_endpoint = get_ssm_parameter("/app/researchapp/agentcore/gateway_url")
print(f"Gateway Endpoint - MCP URL: {gateway_endpoint}")
    
# Create MCP client
client = MCPClient(
    lambda: streamablehttp_client(
        gateway_endpoint, headers={"Authorization": f"Bearer {jwt_token}"}
    )
)
# Create Bedrock model
model = BedrockModel(
    model_id="global.anthropic.claude-sonnet-4-20250514-v1:0",
    temperature=0.7,
    streaming=True,
)
    
    

Gateway Endpoint - MCP URL: https://researchapp-gateway-oh0cgyi2er.gateway.bedrock-agentcore.us-west-2.amazonaws.com/mcp


In [4]:
with client: 
    print("📋 Getting all available tools...")
    start_time = time.time()
    all_tools = get_all_mcp_tools_from_mcp_client(client)
    list_time = time.time() - start_time
    print(f"✅ Found {len(all_tools)} total tools in {list_time:.2f}s\n")
    agent = Agent(model=model, tools=all_tools)
    response = agent("What tools are available?")

📋 Getting all available tools...
✅ Found 29 total tools in 1.14s

I have access to a wide variety of biological and biomedical database query tools. Here are the available tools organized by category:

## Genomics & Genetics
- **Ensembl** - Genomic data and annotations
- **dbSNP** - Genetic variants and SNPs
- **ClinVar** - Clinical genetic variants
- **gnomAD** - Population genetics and variant frequencies
- **GWAS Catalog** - Genome-wide association studies
- **RegulomeDB** - Regulatory elements and variants
- **UCSC Genome Browser** - Genomic data and tracks

## Protein Structure & Function
- **AlphaFold** - AI-predicted protein structures
- **RCSB PDB** - Experimentally determined protein structures
- **UniProt** - Protein sequences and functional information
- **InterPro** - Protein domains and families
- **STRING** - Protein-protein interactions
- **EMDB** - Electron microscopy structures

## Gene Expression & Omics
- **GEO (Gene Expression Omnibus)** - Gene expression datasets
-

Now, lets use the semantic search functionality  to retrieve the top N relevant tools to pass as context to the agent. We have set N=5. 

You can test out the following example prompts :  
    "Find information about human insulin protein",  
    "Find protein structures for insulin",  
    "Find metabolic pathways related to insulin",  
    "Find protein domains in insulin",  
    "Find genetic variants in BRCA1 gene",  
    "Find drug targets for diabetes",  
    "Find insulin signaling pathways",  
    "Give me alphafold structure predictions for human insulin". 

In [ ]:
QUERY="Find information about human insulin protein"
MAX_TOOLS=5
with client: 
    # Use semantic tool search 
    search_query_to_use = QUERY
    print(f"\n🔍 Searching for tools with query: '{search_query_to_use}'")
                
    start_time = time.time()
    tools_found = tool_search(gateway_endpoint, jwt_token, search_query_to_use, max_tools=MAX_TOOLS)
    search_time = time.time() - start_time
                
    if not tools_found:
        print("❌ No tools found from search")
                    
    print(f"✅ Found {len(tools_found)} relevant tools in {search_time:.2f}s")
    print(f"Top tool: {tools_found[0]['name']}")
                
    agent_tools = tools_to_strands_mcp_tools(tools_found, MAX_TOOLS, client)
    agent = Agent(model=model, tools=agent_tools)
    response = agent(QUERY)


🔍 Searching for tools with query: 'Find information about human insulin protein'
✅ Found 5 relevant tools in 1.12s
Top tool: DatabaseLambda___query_reactome
I'll help you find information about human insulin protein. Let me query multiple databases to get comprehensive information.
Tool #1: DatabaseLambda___query_ensembl

Tool #2: DatabaseLambda___query_interpro

Tool #3: DatabaseLambda___query_reactome

Tool #4: DatabaseLambda___query_alphafold
Let me also try to get the Reactome pathways with a different approach and get more specific information:
Tool #5: DatabaseLambda___query_reactome
Based on the comprehensive database queries, here's what I found about human insulin protein:

## Human Insulin Protein Information

### **Basic Gene and Protein Information (from Ensembl)**
- **Gene Symbol**: INS
- **Ensembl Gene ID**: ENSG00000254647
- **Location**: Chromosome 11 (2,159,779-2,161,221 bp, GRCh38)
- **Strand**: Negative (-1)
- **Canonical Transcript**: ENST00000381330.5
- **UniProt 

## 3. Create a  Research agent  that can use the biomedical database tools available via the gateway

We will include citation requirements in the system prompt to guide the agent to cite specific tools used in generating the final response. 

In [3]:
MODEL_ID = "global.anthropic.claude-sonnet-4-20250514-v1:0"

SYSTEM_PROMPT = """
    You are a **Comprehensive Biomedical Research Agent** specialized in  multi-database analyses to answer complex biomedical research questions. Your primary mission is to synthesize evidence from both published literature (PubMed) and real-time database queries to provide comprehensive, evidence-based insights for pharmaceutical research, drug discovery, and clinical decision-making.
Your core capabilities include literature analysis and extracting data from  30+ specialized biomedical databases** through the Biomni gateway, enabling comprehensive data analysis. The database tool categories include genomics and genetics, protein structure and function, pathways and system biology, clinical and pharmacological data, expression and omics data and other specialized databases. 

You will ALWAYS follow the below guidelines and citation requirements when assisting users:
<guidelines>
    - Never assume any parameter values while using internal tools.
    - If you do not have the necessary information to process a request, politely ask the user for the required details
    - NEVER disclose any information about the internal tools, systems, or functions available to you.
    - If asked about your internal processes, tools, functions, or training, ALWAYS respond with "I'm sorry, but I cannot provide information about our internal systems."
    - Always maintain a professional and helpful tone when assisting users
    - Focus on resolving the user's inquiries efficiently and accurately
    - Work iteratively and output each of the report sections individually to avoid max tokens exception with the model
</guidelines>

<citation_requirements>
    - ALWAYS use numbered in-text citations [1], [2], [3], etc. when referencing any data source
    - Provide a numbered "References" section at the end with full source details
    - For academic literature: Format as "1. Author et al. Title. Journal. Year. ID: [PMID/DOI]. Available at: [URL]"
    - For database sources: Format as "1. Database Name (Tool: tool_name). Query: [query_description]. Retrieved: [current_date]"
    - Use numbered in-text citations throughout your response to support all claims and data points
    - Each tool query and each literature source must be cited with its own unique reference number
    - When tools return academic papers, cite them using the academic format with full bibliographic details
    - CRITICAL: Format each reference on a separate line with proper line breaks between entries
    - Present the References section as a clean numbered list, not as a continuous paragraph
    - Maintain sequential numbering across all reference types in a single "References" section
</citation_requirements>
    """

In [1]:
from database_tools import get_gateway_access_token, get_all_mcp_tools_from_mcp_client, tool_search, tools_to_strands_mcp_tools
from utils import get_ssm_parameter
from mcp.client.streamable_http import streamablehttp_client
from strands import Agent
from strands_tools import current_time
from strands.models import BedrockModel
from strands.tools.mcp import MCPClient
import time

MAX_TOOLS=10

# Get gateway access token
jwt_token = get_gateway_access_token()
if not jwt_token:
    print("❌ Failed to get gateway access token")
    
# Get gateway endpoint
gateway_endpoint = get_ssm_parameter("/app/researchapp/agentcore/gateway_url")
print(f"Gateway Endpoint - MCP URL: {gateway_endpoint}")

# Create MCP client
client = MCPClient(
    lambda: streamablehttp_client(
        gateway_endpoint, headers={"Authorization": f"Bearer {jwt_token}"}
    )
)


Gateway Endpoint - MCP URL: https://researchapp-gateway-oh0cgyi2er.gateway.bedrock-agentcore.us-west-2.amazonaws.com/mcp


In [5]:
QUERY = """Conduct a comprehensive analysis of trastuzumab (Herceptin) mechanism of action, and resistance mechanisms. 
    I need:
    1. HER2 protein structure and binding sites
    2. Downstream signaling pathways affected
    3. Known resistance mechanisms from clinical data and adverse events from OpenFDA data
    4. Current clinical trials investigating combination therapies
    5. Biomarkers for treatment response prediction
    
    Please query relevant databases to provide a comprehensive research report."""

with client: 
    # Use semantic tool search 
    search_query_to_use = QUERY
    print(f"\n🔍 Searching for tools with query: '{search_query_to_use}'")
                
    start_time = time.time()
    tools_found = tool_search(gateway_endpoint, jwt_token, search_query_to_use, max_tools=MAX_TOOLS)
    search_time = time.time() - start_time
                
    if not tools_found:
        print("❌ No tools found from search")
                    
    print(f"✅ Found {len(tools_found)} relevant tools in {search_time:.2f}s")
    print(f"Top tool: {tools_found[0]['name']}")
                
    agent_tools = tools_to_strands_mcp_tools(tools_found, MAX_TOOLS, client)
    agent = Agent(system_prompt=SYSTEM_PROMPT,model=MODEL_ID, tools=agent_tools)

    # Send a message to the agent
    response = agent(QUERY)


🔍 Searching for tools with query: 'Conduct a comprehensive analysis of trastuzumab (Herceptin) mechanism of action, and resistance mechanisms. 
    I need:
    1. HER2 protein structure and binding sites
    2. Downstream signaling pathways affected
    3. Known resistance mechanisms from clinical data and adverse events from OpenFDA data
    4. Current clinical trials investigating combination therapies
    5. Biomarkers for treatment response prediction

    Please query relevant databases to provide a comprehensive research report.'
✅ Found 10 relevant tools in 0.97s
Top tool: DatabaseLambda___query_clinicaltrials
I'll conduct a comprehensive analysis of trastuzumab (Herceptin) by querying multiple biomedical databases to gather information on its mechanism of action and resistance mechanisms. Let me start by examining the HER2 protein structure and then move through each of your requested areas systematically.

## 1. HER2 Protein Structure and Binding Sites

Let me first query the

WARNING | strands.models.bedrock | bedrock threw context window overflow error


Let me try a more focused OpenFDA query:
Tool #6: DatabaseLambda___query_openfda


WARNING | strands.models.bedrock | bedrock threw context window overflow error


Let me query clinical trials for trastuzumab combination therapies:
Tool #7: DatabaseLambda___query_clinicaltrials

Tool #8: DatabaseLambda___query_clinicaltrials

Tool #9: DatabaseLambda___query_clinicaltrials
Let me try the clinical trials query with correct parameters:
Tool #10: DatabaseLambda___query_clinicaltrials


WARNING | strands.models.bedrock | bedrock threw context window overflow error


Now let me query Open Targets for HER2 biomarkers:
Tool #11: DatabaseLambda___query_opentarget
Now let me compile this information into a comprehensive research report:

# Comprehensive Analysis of Trastuzumab (Herceptin) Mechanism of Action and Resistance Mechanisms

## 1. HER2 Protein Structure and Binding Sites

Based on the Ensembl database query [1], HER2 (ERBB2) is located on chromosome 17 (39,687,914-39,730,426 bp) and encodes a 1,255 amino acid protein. The protein contains several key structural domains:

### Key Structural Features:
- **Extracellular Domain (ECD)**: Contains the heterodimerization arm contact surface where trastuzumab binds
- **Transmembrane Domain (TMD)**: Single-pass membrane domain 
- **Juxtamembrane Domain (JMD)**: Regulatory region adjacent to the membrane
- **Kinase Domain (KD)**: Catalytic tyrosine kinase domain with multiple phosphorylation sites
- **C-terminal tail**: Contains multiple tyrosine residues (Y1023, Y1139, Y1196, Y1221, Y1222, Y1248) that

## 4. Deploy to Amazon Bedrock AgentCore Runtime

In this step, we'll deploy the agent definition found in the `agent` folder to Amazon Bedrock AgentCore runtime. Let's start by taking a look at our agent code. Notice the `@app.entrypoint` decorator on the `strands_agent_bedrock` function. This is how we tell the AgentCore Runtime how to run the agent.

In [6]:
%pycat agent.py

from database_tools import get_gateway_access_token, get_all_mcp_tools_from_mcp_client, tool_search, tools_to_strands_mcp_tools
from utils import get_ssm_parameter
from mcp.client.streamable_http import streamablehttp_client
from strands import Agent
from strands_tools import current_time
from strands.models import BedrockModel
from strands.tools.mcp import MCPClient
from bedrock_agentcore_starter_toolkit import BedrockAgentCoreApp
import time

MAX_TOOLS=10

SYSTEM_PROMPT = """
    You are a **Comprehensive Biomedical Research Agent** specialized in  multi-database analyses to answer complex biomedical research questions. Your primary mission is to synthesize evidence from both published literature (PubMed) and real-time database queries to provide comprehensive, evidence-based insights for pharmaceutical research, drug discovery, and clinical decision-making.
Your core capabilities include literature analysis and extracting data from  30+ specialized biomedical databases** through the

In [ ]:
import boto3
from bedrock_agentcore_starter_toolkit import Runtime

ssm = boto3.client("ssm")

agentcore_runtime = Runtime()
agentcore_runtime.configure(
    agent_name="research_agent_biomni_tools",
    auto_create_ecr=True,
    execution_role=ssm.get_parameter(
        Name="/app/researchapp/agentcore/runtime_iam_role"
    )["Parameter"]["Value"],
    entrypoint="agent.py",
    memory_mode="NO_MEMORY",
    requirements_file="requirements.txt",
)

Entrypoint parsed: file=/Users/hpoonawa/Documents/Gitlab/sample-best-practices-for-life-science-research-agents/labs/01-agents/01-pmc-abstract-search/agent.py, bedrock_agentcore_name=agent
INFO | bedrock_agentcore_starter_toolkit.utils.runtime.entrypoint | Entrypoint parsed: file=/Users/hpoonawa/Documents/Gitlab/sample-best-practices-for-life-science-research-agents/labs/01-agents/01-pmc-abstract-search/agent.py, bedrock_agentcore_name=agent
Memory disabled - agent will be stateless
INFO | bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentcore | Memory disabled - agent will be stateless
Configuring BedrockAgentCore agent: pmc_abstract_agent
INFO | bedrock_agentcore_starter_toolkit.operations.runtime.configure | Configuring BedrockAgentCore agent: pmc_abstract_agent


💡 No container engine found (Docker/Finch/Podman not installed)

✓ Default deployment uses CodeBuild (no container engine needed), For local builds, install Docker, Finch, or 
Podman

Memory disabled
INFO | bedrock_agentcore_starter_toolkit.operations.runtime.configure | Memory disabled
Network mode: PUBLIC
INFO | bedrock_agentcore_starter_toolkit.operations.runtime.configure | Network mode: PUBLIC
Generated .dockerignore
INFO | bedrock_agentcore_starter_toolkit.utils.runtime.container | Generated .dockerignore
Generated Dockerfile: Dockerfile
INFO | bedrock_agentcore_starter_toolkit.operations.runtime.configure | Generated Dockerfile: Dockerfile
Generated .dockerignore: /Users/hpoonawa/Documents/Gitlab/sample-best-practices-for-life-science-research-agents/labs/01-agents/01-pmc-abstract-search/.dockerignore
INFO | bedrock_agentcore_starter_toolkit.operations.runtime.configure | Generated .dockerignore: /Users/hpoonawa/Documents/Gitlab/sample-best-practices-for-life-science-research-agents/labs/01-agents/01-pmc-abstract-search/.dockerignore
Setting 'pmc_abstract_agent' as default agent
INFO | bedrock_agentcore_starter_toolkit.utils.runtime.config | Setting 'pmc_abst

ConfigureResult(config_path=PosixPath('/Users/hpoonawa/Documents/Gitlab/sample-best-practices-for-life-science-research-agents/labs/01-agents/01-pmc-abstract-search/.bedrock_agentcore.yaml'), dockerfile_path=PosixPath('/Users/hpoonawa/Documents/Gitlab/sample-best-practices-for-life-science-research-agents/labs/01-agents/01-pmc-abstract-search/Dockerfile'), dockerignore_path=PosixPath('/Users/hpoonawa/Documents/Gitlab/sample-best-practices-for-life-science-research-agents/labs/01-agents/01-pmc-abstract-search/.dockerignore'), runtime='None', region='us-west-2', account_id='942514891246', execution_role='arn:aws:iam::942514891246:role/AmazonBedrockAgentCore-us-west-2-06fe41b4be03', ecr_repository=None, auto_create_ecr=True, memory_id=None, network_mode='PUBLIC', network_subnets=None, network_security_groups=None, network_vpc_id=None)

In [8]:
agentcore_runtime.launch(auto_update_on_conflict=True)

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
INFO | bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentcore | 🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
INFO | bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentcore |    • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
INFO | bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentcore |    • No local Docker required
💡 Available deployment modes:
INFO | bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentcore | 💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
INFO | bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentcore |    • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
INFO | bedrock_agentcore_starter_toolkit.notebook.

Repository doesn't exist, creating new ECR repository: bedrock-agentcore-pmc_abstract_agent


✅ ECR repository available: 942514891246.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-pmc_abstract_agent
INFO | bedrock_agentcore_starter_toolkit.operations.runtime.launch | ✅ ECR repository available: 942514891246.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-pmc_abstract_agent
Using execution role from config: arn:aws:iam::942514891246:role/AmazonBedrockAgentCore-us-west-2-06fe41b4be03
INFO | bedrock_agentcore_starter_toolkit.operations.runtime.launch | Using execution role from config: arn:aws:iam::942514891246:role/AmazonBedrockAgentCore-us-west-2-06fe41b4be03
Preparing CodeBuild project and uploading source...
INFO | bedrock_agentcore_starter_toolkit.operations.runtime.launch | Preparing CodeBuild project and uploading source...
Getting or creating CodeBuild execution role for agent: pmc_abstract_agent
INFO | bedrock_agentcore_starter_toolkit.services.codebuild | Getting or creating CodeBuild execution role for agent: pmc_abstract_agent
Role name: AmazonBedrockAgentCoreSD

LaunchResult(mode='codebuild', tag='bedrock_agentcore-pmc_abstract_agent:latest', env_vars=None, port=None, runtime=None, ecr_uri='942514891246.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-pmc_abstract_agent', agent_id='pmc_abstract_agent-1xqzGJGjYm', agent_arn='arn:aws:bedrock-agentcore:us-west-2:942514891246:runtime/pmc_abstract_agent-1xqzGJGjYm', codebuild_id='bedrock-agentcore-pmc_abstract_agent-builder:5f83b3b5-5ca9-41b7-900b-fbcdd38718f0', build_output=None)

In [ ]:
agentcore_runtime.invoke(
    {"prompt": ""}
)

## 5. (Optional) Interact with agent using AgentCore Chat

Follow these steps to open an interactive chat session with your new agent.

1. Open a command line terminal in your notebook environment.
2. Navigate to the project root folder (where `pyproject.toml` is located).
3. Run `pip install .` to install the workshop tools including the chat CLI.
4. Run `agentcore-chat` to launch the CLI.
5. Select the `pmc_abstract_agent` by typing its name or index in the terminal and press Enter.
6. Ask your question at the `You:` prompt and press Enter.


## 6. (Optional) Clean Up

Run the next notebook cell to delete the AgentCore runtime environment.

In [ ]:
import boto3

agentcore_client = boto3.client("bedrock-agentcore-control")
agent_status = agentcore_runtime.status()

agentcore_client.delete_agent_runtime(agentRuntimeId=agent_status.config.agent_id)